In [2]:
using HTTP, JSON, PrettyTables, JLD, DotEnv,  Random, Dates
institutions = load("institutions.jld")["institutions"]
academic_list = load("academic_list.jld")["academic_list"]
adjacency = load("adjacency_2023.jld")["out"];

In [5]:
C = zeros(Int32,12,length(institutions))
T = zeros(Int32, length(academic_list), 5);

In [7]:
# fill in the hard coded sinks
est_alloc = load("est_alloc.jld")["est_alloc"];
for i in length(academic_list)+1:length(institutions)
    C[est_alloc[i], i] = 1
end

In [16]:
# sinks have been hard coded - start by assigning all academic institutions to community 1
for i in 1:length(academic_list)
    T[i,1] = 1
    C[1,i] = 1
end
#now all academic institutions are in the same tier

In [17]:
C*adjacency*T

12×5 Matrix{Int32}:
 650  0  0  0  0
   0  0  0  0  0
   0  0  0  0  0
   0  0  0  0  0
   0  0  0  0  0
 153  0  0  0  0
 146  0  0  0  0
  81  0  0  0  0
  12  0  0  0  0
 266  0  0  0  0
  52  0  0  0  0
 236  0  0  0  0

In [140]:
function likelihood(adjacency)
    t = sum(adjacency,dims = 1)
    c = sum(adjacency, dims=2)
    l = 0.0
    for i in 1:12,j in 1:5
        l +=  -adjacency[i,j]*(log(max(adjacency[i,j]/(max(c[i]*t[j],1)),.0001)) - 1)
    end
    return l
end
    

likelihood (generic function with 1 method)

In [141]:
likelihood(C*adjacency*T)

13366.90822170357

In [142]:
function best_alloc(C,T,adjacency, iterations = 10) 
    start = likelihood(C*adjacency*T)
    sq = [C, T, start]
    println("Starting value: ", start)
    n = 0
    c = size(C)
    t = size(T)
    while n < iterations
        shuffle = []
        to_tier = []
        new_C = zeros(Int32,c[1],c[2])
        new_T = zeros(Int32, t[1],t[2])
        for j in 1:10
            push!(shuffle,rand(1:t[1]))
            push!(to_tier, rand(1:t[2]))
        end
        for i in 1:c[2], j in 1:c[1]
            index = findfirst(x -> x == i, shuffle)
            if !isnothing(index)
                if j == to_tier[index] 
                    new_C[j, i] = 1
                    new_T[i, j] = 1
                else
                    new_C[j, i] = 0
                    if j < t[2] + 1
                        new_T[i, j] = 0
                    end
                end
            else
                new_C[j,i] = sq[1][j,i]
                if i < t[1] + 1
                    if j < 6
                        new_T[i, j] = sq[2][i,j]
                    end
                end
            end
        end
        new_l = likelihood(new_C*adjacency*new_T)
        
        if new_l < sq[3]
            println("Iteration ", n, ": " , new_l)
            sq[3] = new_l
            sq[1] = new_C
            sq[2] = new_T
        end
        n += 1
    end
                
    return sq
end
        

best_alloc (generic function with 2 methods)

In [194]:
sq = best_alloc(C,T,adjacency, 150000);


Starting value: 13366.90822170357
Iteration 0: 13356.490785577125
Iteration 1: 13355.328147489257
Iteration 2: 13346.989102346612
Iteration 3: 13341.746931407557
Iteration 4: 13341.62144038443
Iteration 6: 13336.919523270219
Iteration 7: 13331.57374329036
Iteration 8: 13329.549141357244
Iteration 9: 13326.234059856775
Iteration 13: 13325.78169978515
Iteration 20: 13324.71991125064
Iteration 22: 13321.251721339768
Iteration 23: 13319.256492713148
Iteration 25: 13318.778388222137
Iteration 28: 13313.05866184413
Iteration 29: 13312.1739232213
Iteration 32: 13311.23896413938
Iteration 33: 13308.970287686956
Iteration 36: 13303.888538052883
Iteration 39: 13301.496134304258
Iteration 44: 13300.082325362138
Iteration 52: 13299.747364138228
Iteration 58: 13299.599882203398
Iteration 59: 13299.020000023067
Iteration 60: 13298.584463302168
Iteration 61: 13294.616431649809
Iteration 62: 13294.334850719477
Iteration 70: 13289.64804222858
Iteration 75: 13286.074582267258
Iteration 77: 13283.5616319

In [147]:
sum(sq[1])

991

In [195]:
sq[1]*adjacency*sq[2]

12×5 Matrix{Int32}:
 102  16  13   0   25
  20  76  15   0   28
   0  20  56  12    6
   0   7   2  38   20
  18  10  34   1  131
  43  26   5  15   64
  14  32  24   5   71
  46  21   1   5    8
   4   3   0   0    5
  38  53  34  77   64
  18   0  11   0   23
  15  67  47  45   62

In [198]:
for i in 1:size(adjacency,2)
    if sq[2][i,1] == 1
        println(institutions[i])
    end
end

Aarhus University
Aix-Marseille Université
Aligarh Muslim University
Berlin School of Economics
Bocconi University
Brown University
Brunel University London
CEMFI
CERGE-EI (Center for Economic Research and Graduate Education – Economics Institute)
CY Cergy Paris Université
Center for Research in Economics and Statistics (CREST)
Central University of Finance and Economics
Dalhousie University
Durham University
Erasmus University Rotterdam
Erciyes University
European University Institute
Gebze Technical University
George Washington University
Ghent University
Glasgow Caledonian University
Goethe-Universität Frankfurt am Main
Grenoble Ecole de Management
HEC Paris
Hebrew University of Jerusalem
Howard University
Humboldt University Berlin
Johannes Gutenberg-University Mainz
Johns Hopkins University
Katholieke Universiteit Leuven
King's College London
Linnéuniversitetets
Ludwig-Maximilians-Universitat München (LMU) (University of Munich)
Lund University
Nigerian Defence Academy
Radboud Uni

In [197]:
sum(sq[2], dims = 1)

1×5 Matrix{Int64}:
 91  93  64  84  80